In [1]:
import os
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import pandas as pd
import numpy as np

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "service_acc/alpine-sentry-349911-6e79cefcab88.json"

In [3]:
def predict_tabular_classification_sample(project, endpoint_id,instance_dict,location,api_endpoint):
    
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    
    # for more info on the instance schema, please use get_model_sample.py
    # and look at the yaml found in instance_schema_uri
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
#     print(f"Brand + Model Name: {instance_dict['brand']} {instance_dict['model']}")
#     print("deployed_model_id:", response.deployed_model_id)
    
    predictions = response.predictions
    for prediction in predictions:
        result = dict(prediction)
#         print("prediction:", result)
    return response, result

### 

### Get Prediction

In [4]:
df = pd.read_csv('data/structured-data/Mobile_price_range_cls.csv')
n = 10
data = df.sample(n)
instances = []
for i in range(n):
    instance = data.iloc[i].to_dict()
    for k,v in instance.items():
        instance[k] = str(v)
    
    instances.append(instance)

In [10]:
for i in range(n):
    response, result = predict_tabular_classification_sample(project="797761434506",
                                                   endpoint_id="3393919716060823552",
                                                   location="us-west2",
                                                   api_endpoint= "us-west2-aiplatform.googleapis.com",
                                                   instance_dict = instances[i])

    classes = result['classes']
    proba = np.array(result['scores'])
    pred = classes[np.argmax(proba)]
    
    print("Actual Price Range:",instances[i]['price_range'], 
          "Predicted Price Range:",pred,": ",np.round(proba[np.argmax(proba)],4))
    print()
    

Actual Price Range: 3.0 Predicted Price Range: 3 :  1.0

Actual Price Range: 2.0 Predicted Price Range: 2 :  0.9981

Actual Price Range: 3.0 Predicted Price Range: 3 :  0.9965

Actual Price Range: 0.0 Predicted Price Range: 0 :  0.9995

Actual Price Range: 1.0 Predicted Price Range: 1 :  0.999

Actual Price Range: 2.0 Predicted Price Range: 2 :  0.9487

Actual Price Range: 0.0 Predicted Price Range: 0 :  1.0

Actual Price Range: 1.0 Predicted Price Range: 1 :  0.9988

Actual Price Range: 2.0 Predicted Price Range: 2 :  0.9338

Actual Price Range: 2.0 Predicted Price Range: 2 :  0.9828



In [11]:
from typing import Dict
from google.cloud import aiplatform_v1beta1

In [12]:
def explain_tabular_sample(project: str,endpoint_id: str,instance_dict: Dict,location: str = "us-central1",
                           api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform_v1beta1.PredictionServiceClient(client_options=client_options)
    
    # The format of each instance should conform to the deployed model's prediction input schema.
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    
    # tabular models do not have additional parameters
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.explain(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    explanations = response.explanations
    for explanation in explanations:
        print(" explanation")
        # Feature attributions.
        attributions = explanation.attributions
        for attribution in attributions:
            print("  attribution")
            print("   baseline_output_value:", attribution.baseline_output_value)
            print("   instance_output_value:", attribution.instance_output_value)
            print("   output_display_name:", attribution.output_display_name)
            print("   approximation_error:", attribution.approximation_error)
            print("   output_name:", attribution.output_name)
            output_index = attribution.output_index
            for output_index in output_index:
                print("   output_index:", output_index)
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

In [13]:
explain_tabular_sample(project="797761434506",
                       endpoint_id="3393919716060823552",
                       location="us-west2",
                       api_endpoint= "us-west2-aiplatform.googleapis.com",
                       instance_dict = instances[0])

response
 deployed_model_id: 830104890691813376
 explanation
  attribution
   baseline_output_value: 2.4056191705312813e-07
   instance_output_value: 1.0
   output_display_name: 3
   approximation_error: 0.000555801154407209
   output_name: scores
   output_index: 3
 prediction: {'scores': [2.968311614669329e-24, 1.096669934986271e-17, 4.75908130681546e-08, 1.0], 'classes': ['0', '1', '2', '3']}
